Processando páginas:   0%|          | 0/51 [00:00<?, ?it/s]2024-06-07 16:14:54,741 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'get_text'
2024-06-07 16:14:54,745 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:14:54,749 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:14:54,751 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:14:54,755 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:14:54,759 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:14:54,764 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:14:54,765 - ERROR - Erro ao processar o imóvel: 'NoneType' object has no attribute 'find_next_sibling'
2024-06-07 16:

ValueError: could not convert string to float: 'R$9990'